# Prepare data

First we will create a dictionary of Semcor words, and look at them and their frequencies.


Next, we want to create a dataset of a subsample of semcor. We want to remove the most common and least common words

We also want to limit to 30 examples of each sense of a word.

So, we begin iterating through a randomly shuffled semcor. For each word, we throw it out if it is too frequent or too infreqient. Then, we look at the senses.



At the end, we store a list of all of the words we've collected. For each item in the dictionary, we should know:
- the number of tokens
- the wordnet senses
- a list of the semcor sentence indices of the tokens of each word. 


In [1]:
import sys
sys.path.append("../lib/")

from nltk.corpus import semcor
from nltk.tree import Tree
import itertools
import random
import pandas as pd
import torch
from bert import *
import csv
from nltk.corpus.reader.wordnet import Lemma
from nltk.corpus import wordnet as wn
import inflect
import os

from scipy.stats import spearmanr, pearsonr
import re

bert = BERTBase()


INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpmtlnkm33
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [2]:
p = inflect.engine()
p.plural("mouse")

'mice'

In [3]:
"""
load semcor stats
"""

# uncomment for whole dataset
sents = semcor.sents()
tagged_sents = semcor.tagged_sents( tag = ' sem ' )
words = semcor.words()


##########
# DEBUG ONLY
############

# tagged_sents = semcor.tagged_sents( tag = ' sem ' )[:20]
# sents = semcor.sents()[:20]
# words = semcor.words()[1000]





In [4]:
"""
create a dictionary of tokens in semcor with counts
"""
from collections import Counter

semcor_lexicon = Counter()

for word in words:
    word = word.lower()
    semcor_lexicon[word] +=1

semcor_lexicon

Counter({'the': 49074,
         'fulton': 14,
         'county': 125,
         'grand': 40,
         'jury': 55,
         'said': 1412,
         'friday': 59,
         'an': 2648,
         'investigation': 39,
         'of': 24954,
         'atlanta': 27,
         "'s": 4037,
         'recent': 123,
         'primary': 65,
         'election': 72,
         'produced': 62,
         '``': 6169,
         'no': 1520,
         'evidence': 133,
         "''": 6097,
         'that': 7541,
         'any': 957,
         'irregularities': 7,
         'took': 293,
         'place': 422,
         '.': 34716,
         'further': 149,
         'in': 15012,
         'term': 52,
         'end': 265,
         'presentments': 1,
         'city': 345,
         'executive': 37,
         'committee': 116,
         ',': 40614,
         'which': 2495,
         'had': 3534,
         'over-all': 19,
         'charge': 69,
         'deserves': 13,
         'praise': 12,
         'and': 19960,
         'thanks':

In [5]:
"""
little script on semcor_lexicon to see where we should cut off most and least frequent
"""

print(semcor_lexicon.most_common(200))
n = 30000
print(semcor_lexicon.most_common()[:-n-1:-1])

# we want to keep words with a count < 600

# and with a count greater than > 10 (which is knocking off the l)

[('the', 49074), (',', 40614), ('.', 34716), ('of', 24954), ('and', 19960), ('to', 18089), ('a', 16399), ('in', 15012), ('that', 7541), ('is', 7228), ('he', 6812), ('was', 6773), ('for', 6574), ('it', 6375), ('``', 6169), ("''", 6097), ('with', 5145), ('as', 4953), ('on', 4818), ('his', 4604), ('be', 4552), ("'s", 4037), ('at', 3765), ('by', 3688), ('this', 3639), ('i', 3589), ('had', 3534), ('not', 3247), ('are', 3171), ('but', 3076), ('from', 3032), ('-', 2967), ('or', 2917), ('have', 2811), ('they', 2759), ('an', 2648), ('you', 2629), ('which', 2495), ('one', 2359), ('were', 2319), ('all', 2121), ('would', 2052), ('there', 2019), (';', 1997), ('we', 1972), ('she', 1968), ('their', 1926), ('her', 1909), ('has', 1857), ('him', 1801), ('?', 1715), ('been', 1714), ('when', 1629), ('will', 1611), ('more', 1592), ('who', 1580), ('if', 1545), (')', 1535), ('(', 1524), ('no', 1520), ('out', 1488), ('said', 1412), ("n't", 1407), ('so', 1384), ('can', 1379), ('what', 1377), ('do', 1358), ('up

In [6]:
"""
get basic semcor stats
"""
print("number of sentences:")
print(len(sents))
print("number of tokens:")
print(len(words))
print("number of types:")
print(len(semcor_lexicon))

number of sentences:
37176
number of tokens:
820411
number of types:
38642


Then, for each word, we find all of the tokens of that word in the corpus, and use the model to predict features for that word.



In [7]:
"""
Next step is to create an index of all of the tokens of a single word form. 
So, we build a data structure with all of the word forms found in semcor. With each word form,
we store a list of all of the sentences containing it.
"""
word_index = {}
sense_index = {}

semcor_indices = list(range(0,len(sents)))
#print(semcor_indices)
random.shuffle(semcor_indices)
#print(semcor_indices)


# go through the dataset sentence by sentence
for random_index in semcor_indices:
    sent = sents[random_index]
    #sentence_id = int(sent.num)
    sentence_id = random_index
    
    # go through each sentence word by word
    for word in sent:
        
    # we need to make sure we are collecting only those tokens which have semcor senses, or we make note of which ones do
    
        # if this is our first time seeing this word, add it to the index and put the sentence id in the entry
        if word not in word_index:
            word = word.lower()
            word_index[word] = {sentence_id}
        # otherwise, add the sentence id to the entry for the word
        else:
            word_index[word].add(sentence_id)
            

In [8]:
"""
let's take a look at it
"""
print(list(word_index.items())[:20])

[('to', {11649, 4231, 12811, 24334, 23696, 14491, 2334, 28579, 32421, 34216, 16937, 2864, 10549, 8376, 7097, 15937, 10563, 3528, 13514, 34380, 3533, 15695, 25682, 30932, 24158, 31969, 9059, 15075, 34023, 12263, 26601, 238, 12400, 28528, 13426, 17398, 20855, 31614}), ('him', {18563, 18318, 6406, 18567, 35592, 31368, 6410, 18827, 31116, 6669, 9869, 33422, 19595, 17553, 12050, 6803, 34171, 24334, 8342, 17815, 11160, 19482, 6170, 20127, 671, 35872, 35743, 23075, 33445, 33702, 24492, 17454, 6448, 19385, 10169, 7739, 16826, 33469, 36668, 14529, 10820, 18758, 16838, 35273, 9162, 35145, 9935, 35152, 33619, 10582, 24534, 36185, 24793, 17883, 6753, 35426, 8291, 34787, 7654, 28263, 36072, 8172, 23790, 10735, 7278, 12400, 16883, 9843, 30581, 24310, 17398, 33914, 2171, 15868, 2686, 18303}), ('they', {33248, 15937, 32421, 23274, 24780, 12464, 12400, 1299, 1620, 35735, 7678}), ('were', {4096, 4099, 4100, 24581, 4104, 8203, 4117, 16408, 16412, 8223, 26655, 4136, 26667, 4141, 4143, 16434, 4151, 22585, 

In [9]:
# render1 = wn.lemma('render.v.07.return')
# render2 = wn.lemma('return.v.01.return')

# """
# importnt point about nltk wordnet lemmas. their representation is confusing so be careful. i think equals or differentequals are implmementd in
# unsuspected ways, because you get issues where they dont act like their display name
# """

# dixt = {str(render1): "foo"}
# dixt[str(render2)] = "bar"
# print(dixt)


# dixt = {str(render1): "foo"}
# dixt[str(render2)] = "bar"
# print(dixt)


In [16]:
"""
Now that we have our word index, we want to construct the evaluation dataset

for each word in the index, we want

We iterate through the words in the dictionary.
if that word has a frequence of  > 600 or < 7, we move on.
If it has the right frequency, we pull the indices of the sentences containing that word
we shuffle these indices and access in random order. 
We go through the shuffled indices,
    and we check if we have collected < 50 of this sense.
    if not, we collect this token for the evaluation dataset

collection means:
    we construct a row of data like
        word form
        word sense
        token
        
        
at the end we save the data in a csv file called 'semcor_wu_palmer_eval_data.csv'
"""

def get_sense_in_tagged_sentence(word, tagged_sentence):
    for chunk in tagged_sentence:

        chunk_string = ' '.join(chunk.leaves())

        """
        if we find the word we're looking for in this chunk,
        and that chunk has a wordnet sense (function words dont)
        then scoop it up

        """            
        if chunk_string.lower() == word:
            #print("found %s" % word)
            #print(chunk.label())

            #wn_lemma = cunk.label()
            if isinstance(chunk.label() , Lemma):
                return chunk.label()
    # if we get to the end of the loop. we didn't find the word we were looking for
    return None


def collect_tokens(word, indices, sents, tagged_sents):
    """
    takes a word and a list of indices
    returns tuples containing 
        word
        sentence_string
        sense
    """
    sense_count = Counter()

    print("collecting tokens for ", word)
    print("indices:   ", indices)
    tokens = []
    
    # indices is a list of all of the sentence ids containing this word
    indices = list(indices)
    # visit these sentences in random order
    random.shuffle(indices)
    for index in indices:
        
        #print(word)
        #print(index)
        tagged_sentence = tagged_sents[index]
        #print(tagged_sentence)
        sentence = sents[index]
        #print(sentence)

        sentence = ' '.join(sentence)
        sense = get_sense_in_tagged_sentence(word, tagged_sentence)
        #print(sense)
        
        if sense is None:
            continue
        
        #print(sense.name())
        
        sense = str(sense)
        if sense_count[sense] <= 25:
            tokens.append((word, sentence, sense))
            sense_count[sense] += 1
    
    print(sense_count.items())
    return tokens, sense_count


def collect_tokens_for_all_words_to_file(path, sense_path, word_index, semcor_lexicon, sents, tagged_sents):
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        with open(sense_path, 'w', newline='') as sensefile:
            sensewriter = csv.writer(sensefile)
        
            for word, indices in word_index.items():
                #print(word)
                #print(indices)

                frequency = semcor_lexicon[word]
                if frequency < 600 and frequency > 7:
                    rows, sense_count = collect_tokens(word, indices, sents, tagged_sents)

                    writer.writerows(rows)                    
                    sensewriter.writerow(sense_count.items())
                

# DEBUG TEST
#collect_tokens('heart', word_index['heart'], sents, tagged_sents)

collect_tokens_for_all_words_to_file('../data/semcor_wu_palmer_eval_data.csv', '../data/semcor_sense_counts.csv', word_index, semcor_lexicon, sents, tagged_sents)

collecting tokens for  same
indices:    {33794, 28674, 15364, 8197, 15370, 14859, 11790, 27152, 16400, 2579, 20500, 5142, 29721, 30233, 32794, 31261, 11295, 4131, 32291, 23590, 12328, 28714, 14381, 33838, 31281, 28721, 15923, 2613, 27704, 2617, 7738, 17108, 4670, 19518, 12352, 27201, 24643, 13379, 19527, 2120, 2121, 33866, 4169, 3147, 29772, 36428, 13903, 7248, 13393, 30801, 22610, 28758, 26710, 33193, 13405, 3168, 14433, 32355, 33380, 11366, 5225, 6762, 16492, 25196, 2158, 2668, 32881, 27250, 12915, 4726, 32887, 28792, 4734, 27776, 32390, 4233, 20108, 31373, 15500, 29838, 11410, 11923, 32915, 17557, 22674, 27796, 671, 17572, 6312, 23726, 15022, 20655, 23732, 15545, 14523, 4796, 27838, 16577, 27331, 32965, 28869, 26823, 15045, 25801, 8905, 5837, 717, 3279, 17104, 17618, 15060, 11476, 16084, 13527, 3288, 4825, 26842, 28891, 15065, 24280, 734, 23766, 3292, 14564, 7909, 7910, 16102, 7399, 14567, 12009, 31983, 33009, 15091, 22775, 5367, 1274, 16636, 31485, 1791, 31999, 28929, 28928, 5381, 

dict_items([("Lemma('bring.v.01.bring')", 26), ("Lemma('bring.v.02.bring')", 19), ("Lemma('bring.v.03.bring')", 19), ("Lemma('bring_oneself.v.01.bring_oneself')", 1), ("Lemma('institute.v.02.bring')", 3), ("Lemma('bring.v.04.bring')", 10), ("Lemma('bring.v.06.bring')", 2), ("Lemma('bring.v.05.bring')", 4), ("Lemma('lower.v.01.bring_down')", 1), ("Lemma('return.v.05.bring_back')", 1), ("Lemma('raise.v.07.bring_up')", 1), ("Lemma('bring_up.v.05.bring_up')", 1), ("Lemma('advance.v.05.bring_forward')", 1), ("Lemma('join.v.02.bring_together')", 1), ("Lemma('fetch.v.02.bring_in')", 1)])
collecting tokens for  clung
indices:    {5412, 13574, 8102, 35307, 4716, 18194, 5012, 36021, 18518, 19703, 36062}
dict_items([("Lemma('cling.v.01.cling')", 5), ("Lemma('cling.v.02.cling')", 2)])
collecting tokens for  men
indices:    {865, 31268, 5604, 31368, 5836, 1326, 12464, 25330, 35863, 21304, 14713, 16695}
dict_items([("Lemma('man.n.03.man')", 1), ("Lemma('man.n.01.man')", 2), ("Lemma('serviceman.n.01.

dict_items([("Lemma('publish.v.02.issue')", 5), ("Lemma('issue.n.01.issue')", 15), ("Lemma('issue.v.02.issue')", 2), ("Lemma('issue.n.04.issue')", 1), ("Lemma('issue.n.02.issue')", 2), ("Lemma('write_out.v.02.issue')", 1), ("Lemma('issue.v.03.issue')", 1)])
collecting tokens for  life
indices:    {36897, 14402, 12263, 1515, 24780, 2708, 36150, 19961, 19482, 13531}
dict_items([("Lemma('life.n.01.life')", 1), ("Lemma('life.n.07.life')", 1), ("Lemma('person.n.01.person')", 1)])
collecting tokens for  restless
indices:    {21920, 13827, 31881, 26415, 20624, 35545, 10555, 5756}
dict_items([("Lemma('restless.s.01.restless')", 2)])
collecting tokens for  boredom
indices:    {32089, 27220, 36518}
dict_items([])
collecting tokens for  began
indices:    {11268, 30215, 30217, 7694, 10257, 18450, 18963, 1530, 35352, 36891, 18461, 36907, 12843, 30256, 8753, 12854, 10806, 30775, 5176, 30778, 16954, 21575, 30280, 10825, 35402, 22091, 10832, 26717, 33890, 15459, 35426, 7791, 31863, 24184, 30330, 35457

dict_items([])
collecting tokens for  dressed
indices:    {36992, 19977, 6285, 29713, 36630, 16663, 9629, 24991, 36272, 35633, 1725, 6852, 6853, 19526, 6221, 7374, 36180, 6868, 7386, 2654, 7777, 36731, 36328, 2665, 6384, 10100, 9718, 9339}
dict_items([("Lemma('dress.v.02.dress')", 6), ("Lemma('dress.v.01.dress')", 6), ("Lemma('appareled.s.01.dressed')", 7), ("Lemma('dress.v.04.dress')", 1)])
collecting tokens for  half
indices:    {29218, 2552, 6952, 14634, 34760, 30510, 34351, 29777, 31346, 4146, 8053, 11064, 18652, 33469}
dict_items([("Lemma('one-half.n.01.half')", 5), ("Lemma('half.s.01.half')", 2)])
collecting tokens for  !
indices:    {34816, 10246, 30727, 30726, 28682, 10251, 36881, 36883, 18461, 36896, 26657, 18470, 26666, 36908, 34869, 34870, 36919, 36923, 36926, 20550, 2121, 36942, 32853, 36953, 36954, 16479, 36973, 24689, 16508, 6269, 18567, 14472, 16521, 18573, 18576, 4248, 6301, 2206, 8357, 8359, 16557, 16558, 16559, 8372, 37046, 8377, 37061, 37062, 14534, 6347, 37068, 6348

dict_items([("Lemma('leadership.n.02.leaders')", 9), ("Lemma('leader.n.01.leader')", 14)])
collecting tokens for  find
indices:    {35872, 5450, 8982, 24310, 14686}
dict_items([("Lemma('find.v.05.find')", 2), ("Lemma('find.v.03.find')", 1), ("Lemma('determine.v.01.find')", 1)])
collecting tokens for  hard
indices:    {20416, 7363, 29582, 3153, 2258, 36051, 19282, 35764, 36382}
dict_items([("Lemma('arduous.s.01.hard')", 1), ("Lemma('hard.a.03.hard')", 1), ("Lemma('hard.a.02.hard')", 1), ("Lemma('hard.r.02.hard')", 1)])
collecting tokens for  win
indices:    {1413, 24840, 652, 24974, 28563, 4635, 22050, 28453, 12965, 28454, 28072, 23849, 23978, 683, 28592, 16305, 28595, 24510, 28612, 28613, 26182, 28615, 5706, 24138, 6989, 23121, 15446, 11993, 20450, 8807, 8042, 23019, 13169, 19954, 24819, 244, 24821, 627}
dict_items([("Lemma('acquire.v.05.win')", 14), ("Lemma('win.v.01.win')", 21), ("Lemma('win.n.01.win')", 1)])
collecting tokens for  local
indices:    {2864, 23274, 15093, 27607}
dict_i

...

dict_items([("Lemma('counterpart.n.02.counterpart')", 1), ("Lemma('counterpart.n.01.counterpart')", 7)])
collecting tokens for  semester
indices:    {33187, 33156, 33098, 33110, 33144, 153, 154, 155}
dict_items([])
collecting tokens for  sank
indices:    {14112, 35104, 33827, 24357, 22983, 8584, 30378, 22960, 10001, 21908, 9556, 36052}
dict_items([("Lemma('sink.v.05.sink')", 2), ("Lemma('sink.v.03.sink')", 2), ("Lemma('sink.v.04.sink')", 1), ("Lemma('sink.v.02.sink')", 2), ("Lemma('slump.v.03.sink')", 2), ("Lemma('sink.v.01.sink')", 3)])
collecting tokens for  locations
indices:    {32353, 32451, 31144, 31295, 24885, 32472, 3449, 12765, 2911}
dict_items([("Lemma('location.n.01.location')", 3)])
collecting tokens for  ballroom
indices:    {22070}
dict_items([])
collecting tokens for  collections
indices:    {26784, 22691, 23145, 32554, 25229, 29399, 22139, 21630}
dict_items([])
collecting tokens for  wardrobe
indices:    {23104, 2698, 694, 22459, 22076, 22077, 17599}
dict_items([("Lemma

dict_items([("Lemma('individually.r.01.separately')", 4)])
collecting tokens for  portrait
indices:    {14401, 19557, 31881, 13135, 1428, 8534, 21111, 7353, 26556, 29405}
dict_items([("Lemma('portrayal.n.01.portrait')", 2), ("Lemma('portrait.n.02.portrait')", 2)])
collecting tokens for  jacques
indices:    {6828}
dict_items([])
collecting tokens for  gordon
indices:    {641}
dict_items([("Lemma('person.n.01.person')", 1)])
collecting tokens for  suffer
indices:    {15678, 18946, 4834, 25890, 5349, 25452, 24749, 32081, 24440, 8382}
dict_items([("Lemma('suffer.v.01.suffer')", 3), ("Lemma('suffer.v.06.suffer')", 2), ("Lemma('suffer.v.05.suffer')", 1), ("Lemma('suffer.v.02.suffer')", 1), ("Lemma('digest.v.03.suffer')", 1), ("Lemma('suffer.v.03.suffer')", 2)])
collecting tokens for  thurber
indices:    {26118}
dict_items([])
collecting tokens for  kohnstamm-negative
indices:    {33252}
dict_items([])
collecting tokens for  rail
indices:    {23492, 34062, 33841, 18226, 10579, 8049, 14905, 29

dict_items([("Lemma('desert.n.01.desert')", 2), ("Lemma('abandon.v.05.desert')", 1), ("Lemma('defect.v.01.desert')", 1)])
collecting tokens for  museums
indices:    {32069, 29318, 1008, 29299, 32630, 31580, 31581, 29278}
dict_items([("Lemma('museum.n.01.museum')", 1)])
collecting tokens for  swallowed
indices:    {36035, 16675, 11397, 25703, 8907, 12753, 34803, 36052, 36726, 13023}
dict_items([("Lemma('swallow.v.01.swallow')", 5), ("Lemma('immerse.v.03.swallow')", 2)])
collecting tokens for  intercourse
indices:    {30757, 30823, 30825, 30762, 30767, 30802, 30804, 30780}
dict_items([])
collecting tokens for  sisters
indices:    {21474, 3595, 16011, 21552, 36369, 14480, 21467}
dict_items([("Lemma('sister.n.01.sister')", 3)])
collecting tokens for  anti-slavery
indices:    {28064, 28065, 28069, 28070, 28073, 28042, 28078, 28055, 28058, 28059}
dict_items([])
collecting tokens for  scream
indices:    {30889, 18986, 37166, 7220, 6293, 6295, 36056, 36221, 6047}
dict_items([("Lemma('scream.n.

dict_items([("Lemma('old.a.01.old')", 1)])
collecting tokens for  proclaim
indices:    {32640, 5281, 32674, 7046, 32648, 9768, 32688, 32657, 32630, 32664, 6843}
dict_items([("Lemma('proclaim.v.02.proclaim')", 3), ("Lemma('proclaim.v.01.proclaim')", 8)])
collecting tokens for  geely
indices:    {33973}
dict_items([])
collecting tokens for  committees
indices:    {22883, 23182, 24598}
dict_items([])
collecting tokens for  strings
indices:    {5412, 26629, 26271}
dict_items([("Lemma('string.n.03.string')", 1)])
collecting tokens for  riverside
indices:    {19892}
dict_items([("Lemma('location.n.01.location')", 1)])
collecting tokens for  appealing
indices:    {11648, 27043, 23267, 36486, 22278, 1167, 30002, 26773, 1749, 1176, 29114, 33791}
dict_items([("Lemma('appealing.a.01.appealing')", 3), ("Lemma('sympathetic.a.04.appealing')", 1)])
collecting tokens for  amy
indices:    {1163}
dict_items([])
collecting tokens for  compulsivity
indices:    {15696, 15665, 15667, 15699, 15701, 15669, 15

dict_items([("Lemma('gate.n.01.gate')", 7)])


In [ ]:
print(len(sents))
print(len(tagged_sents))
print(len(word_index))
print(len(semcor_lexicon))

In [17]:
"""
models and save paths

"""
models = [
    'trained_models/model.plsr.buchanan.allbuthomoyms.5k.300components.500max_iters',
    'trained_models/model.plsr.buchanan.allbuthomoyms.1k.300components.500max_iters',
    #'trained_models/model.plsr.buchanan.allbuthomoyms.glove.300components.300max_iters',
    'trained_models/model.ffnn.buchanan.allbuthomoyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    'trained_models/model.ffnn.buchanan.allbuthomoyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'trained_models/model.ffnn.buchanan.allbuthomoyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    'trained_models/model.modabs.buchanan.allbuthomoyms.5k',
    'trained_models/model.modabs.buchanan.allbuthomoyms.1k',
    #'trained_models/model.modabs.buchanan.allbuthomoyms.glove'
]

In [45]:
"""
Now, we have our dataset that we want to analyze. We just need to do:

for each model we want to evaluate, run the following script:

open the file of data

read it in as a dataframe

for each of the unique words in that dataset

    we calculate pairwise distances between each otoken and every otehr token
    and construct a similarities dataset. 
    
    then we run correlations for that word???
    and store into a file
"""

def lemma_from_string(lemma_string):
    # grabs everything in between (' ') in a string
    # (needed to update from r"'(.*?)'" to deal with cases with quotes in word like o'clock)
    string = re.findall(r"\('(.*?)'\)", lemma_string)[0]
    #print(string)
    lemma = wn.lemma(string)
    return lemma

def make_predictions(df, model, bert):
    predictions = []
    for index, row in df.iterrows():

        #print(row.word_form)
        #print(row.context)

        predicted_vector = model.predict_in_context(row.word_form, row.context, bert)

        predictions.append(predicted_vector)
    return predictions

def run_wu_palmer_analysis(model, df, bert):
    unique_words = df.word_form.unique()[:10]
    
    #run_stats = [0] * len(unique_words)
    run_stats = []

    for i in range(0, len(unique_words)):
        if i % 500 == 0:
            print("processed %s words" % i)
        
        # a dataframe containing all the tokens of this word
        word = unique_words[i]
        word_data = df[df.word_form == word].copy()
        predictions = make_predictions(word_data, model, bert)
        
        word_data['prediction'] = predictions
        
        #print(word_data)

        """

        then we calculate the pairwise distances between all of the vectors, only counting one pair one time

        """
        vals_for_this_word = []

        # pop the first token off the list
        num_toks = len(word_data)
        for i in range(0,num_toks):
            # compare it with each of the other tokens
            # dont have to compare to any earlier
            for j in range(i+1,num_toks):

                #print(df.iloc[i])
                #print(df.iloc[j])

                # calculate cosine similarity between the two vectors
                cos_sim = 1 - cosine(word_data.iloc[i].prediction, word_data.iloc[j].prediction)

                # and wu palmer similarity between the two wn lemmas
                lemma_1 = lemma_from_string(word_data.iloc[i].wn_lemma)
                lemma_2 = lemma_from_string(word_data.iloc[j].wn_lemma)
                synset1 = lemma_1.synset()
                synset2 = lemma_2.synset()
                wup_sim = synset1.wup_similarity(synset2)

                # if we can't compute a distance for these senses / recognize them, discard
                if type(wup_sim) == float:
                    # store this data point into a list
                    vals_for_this_word.append((cos_sim, wup_sim))
                    #all_sense_similarities.append((cos_sim, wup_sim))
        
        
        # run correlation on this word
        sense_similarities = pd.DataFrame.from_records(vals_for_this_word, columns = ["cos_sim", "wup_sim"])
        if sense_similarities is not None:
            n = len(word_data)
            n_senses = len(word_data['wn_lemma'].unique())
            #print(word)
            #print(n)
            #print(sense_similarities)
            pearson, pearson_p, spearman, spearman_p = run_correlation(sense_similarities)
            # store the values for this word into a dataframe
            #run_stats[i] = (word, n, pearson, pearson_p, spearman, spearman_p)
            run_stats.append((word, n, n_senses, pearson, pearson_p, spearman, spearman_p))

    res = pd.DataFrame.from_records(run_stats, columns = ["word", "n", "n_senses", "pearson", "pearson_p", "spearman", "spearman_p"])
    return res

    #print(all_sense_similarities)
    #return all_sense_similarities
        
        
def run_correlation(sense_similarities):
    """
    :sense_similarities: dataframe with columns
        cosine_sims 
        wup_sims
    """
    if sense_similarities is None:
        # not really sure why we're getting none values here it should be impossible
        return (float("nan"), float("nan"), float("nan"), float("nan"))
    elif len(sense_similarities['wup_sim'].unique()) == 1:
        # the correlation will be garbage with a constant y value; skip to avoid warnings
        return (float("nan"), float("nan"), float("nan"), float("nan"))

    if len(sense_similarities) > 1 :
        #print(word)

        cos_sims = sense_similarities['cos_sim']
        wup_sims = sense_similarities['wup_sim']

        pearson, pearson_p = pearsonr(cos_sims, wup_sims )
        #print('Pearsons correlation: %.3f, p-value: %s'  % (pearson, pearson_p))

        spearman, spearman_p = spearmanr(cos_sims, wup_sims )
        #print('Spearmans correlation: %.3f, p-value: %s'  % (spearman, spearman_p))

        return (pearson, pearson_p, spearman, spearman_p)
    return (float("nan"), float("nan"), float("nan"), float("nan"))

def plot_sims():
    cos_sims = sense_similarities['cos_sim']
    wup_sims = sense_similarities['wup_sim']
    plt.scatter(wup_sims, cos_sims)
    plt.title("Wordnet similarity of homonymous senses plotted against cosine similarity of predicted vectors of two tokens in semantic feature space")
    plt.xlabel("Wu and Palmer Similarity")
    plt.ylabel("Cosine Similarity")
    plt.show()
        

df = pd.read_csv('../data/semcor_wu_palmer_eval_data.csv', names = ["word_form", "context", "wn_lemma"])



for save_path in models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    run_stats = run_wu_palmer_analysis(model, df, bert)
    #run_stats = run_correlation(similarities)
    out_path = '../results/semcor_analysis_' + os.path.split(save_path)[1] + '.csv'
    run_stats.to_csv(out_path)
    print(run_stats)


****************************************
*** Evaluating trained_models/model.plsr.buchanan.allbuthomoyms.5k.300components.500max_iters model ***
****************************************
processed 0 words
processed 500 words
processed 1000 words
processed 1500 words
processed 2000 words
processed 2500 words
processed 3000 words
processed 3500 words
processed 4000 words
processed 4500 words
processed 5000 words
processed 5500 words
processed 6000 words
             word   n  n_senses   pearson     pearson_p  spearman  \
0            same  74         4       NaN           NaN       NaN   
1     authorities   2         2       NaN           NaN       NaN   
2         release  17         8  0.583408  8.412725e-10  0.701209   
3            seem  38         2  0.214897  8.661733e-09  0.211622   
4            read  34         8  0.464063  2.629645e-31  0.434260   
...           ...  ..       ...       ...           ...       ...   
6365       shades   3         1       NaN           NaN       

/usr/local/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


processed 500 words
processed 1000 words
processed 1500 words
processed 2000 words
processed 2500 words
processed 3000 words
processed 3500 words
processed 4000 words
processed 4500 words
processed 5000 words
processed 5500 words
processed 6000 words
             word   n  n_senses   pearson  pearson_p  spearman  spearman_p
0            same  74         4       NaN        NaN       NaN         NaN
1     authorities   2         2       NaN        NaN       NaN         NaN
2         release  17         8  0.084225   0.422158  0.017937    0.864498
3            seem  38         2 -0.058357   0.122141  0.033843    0.370270
4            read  34         8  0.179135   0.000020  0.192157    0.000005
...           ...  ..       ...       ...        ...       ...         ...
6365       shades   3         1       NaN        NaN       NaN         NaN
6366        coins   3         1       NaN        NaN       NaN         NaN
6367       script   6         1       NaN        NaN       NaN         NaN

/usr/local/lib/python3.8/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


processed 500 words
processed 1000 words
processed 1500 words
processed 2000 words
processed 2500 words
processed 3000 words
processed 3500 words
processed 4000 words
processed 4500 words
processed 5000 words
processed 5500 words
processed 6000 words
             word   n  n_senses   pearson     pearson_p  spearman  \
0            same  74         4       NaN           NaN       NaN   
1     authorities   2         2       NaN           NaN       NaN   
2         release  17         8 -0.113378  2.792141e-01 -0.020035   
3            seem  38         2       NaN           NaN       NaN   
4            read  34         8  0.539882  9.303029e-44  0.459033   
...           ...  ..       ...       ...           ...       ...   
6365       shades   3         1       NaN           NaN       NaN   
6366        coins   3         1       NaN           NaN       NaN   
6367       script   6         1       NaN           NaN       NaN   
6368   simplicity   4         2       NaN           NaN    

In [ ]:
"""
Now what we need to do is go through each of the words and, 
using our model, predict the features in context for each of the tokens of that word.

We iterate through the stored sentenes containing that token.
- we find the word in the sentence, 
- we store its wordnet sense
- and we predict its features. 

"""




#tagged_sents = semcor.tagged_sents( tag = ' sem ' )



all_sense_similarities = []

for word, sentence_ids in word_index.items():
    wordform_data = []

#     print(word)
#     print(sentence_ids)
    for sentence_id in sentence_ids:
        sentence_string = ' '.join(semcor_indexer[sentence_id]).lower()
        
        # the list returned by nltk is indexed one behind the official id of the sentence in semcor
        tagged_sentence = tagged_sents[sentence_id - 1]
        #print(tagged_sentence)
        
        
        for chunk in tagged_sentence:

            chunk_string = ' '.join(chunk.leaves())
            
            """
            if we find the word we're looking for in this chunk,
            and that chunk has a wordnet sense (function words dont)
            then scoop it up

            """            
            if chunk_string.lower() == word:
                #print("found %s" % word)
                #print(chunk.label())

                #wn_lemma = cunk.label()
                if isinstance(chunk.label() , wordnet.Lemma):
                    try:
                        prediction = model.predict_in_context(word, sentence_string, bert)

                        row = {
                            "wordform": word,
                            "wn_lemma": chunk.label(),
                            "semcor_id": sentence_id,
                            "prediction": prediction,
                        }
                        wordform_data.append(row)
                    except:
                        None
        
        #print(row)
        #raise Exception('not done')
    
    
    """

    When we have all of these for a single wordform, we construct a dataset structured like this

        wn_lemma predicted_feature_vector

    First we should note how many senses there are, and how many tokens of each sense

    """
    df = pd.DataFrame.from_records(wordform_data, columns = ["wordform", "wn_lemma", "semcor_id", "prediction"])  
    #print(df)
    
    num_senses = df['wn_lemma'].value_counts()
    #print(counts)

    """

    then we calculate the pairwise distances between all of the vectors, only counting one pair one time

    """
    vals_for_this_token = []
    
    # pop the first token off the list
    num_toks = len(df)
    for i in range(0,num_toks):
        # compare it with each of the other tokens
        # dont have to compare to any earlier
        for j in range(i+1,num_toks):

            #print(df.iloc[i])
            #print(df.iloc[j])
            
            # calculate cosine similarity between the two vectors
            cos_sim = 1 - cosine(df.iloc[i].prediction, df.iloc[j].prediction)
            
            # and wu palmer similarity between the two wn lemmas
            synset1 = df.iloc[i].wn_lemma.synset()
            synset2 = df.iloc[j].wn_lemma.synset()
            wup_sim = synset1.wup_similarity(synset2)
            
            # if we can't compute a distance for these senses / recognize them, discard
            if type(wup_sim) == float:
                # store this data point into a list
                vals_for_this_token.append((cos_sim, wup_sim))
                all_sense_similarities.append((cos_sim, wup_sim))
    
    # store the values for this word into a dataframe
    
    sense_similarities = pd.DataFrame.from_records(vals_for_this_token, columns = ["cos_sim", "wup_sim"])
    # print(sense_similarities)
    
    
    if len(sense_similarities) > 1 :
        print(word)
        
        cos_sims = sense_similarities['cos_sim']
        wup_sims = sense_similarities['wup_sim']
    
        corr, p = pearsonr(cos_sims, wup_sims )
        print('Pearsons correlation: %.3f, p-value: %s'  % (corr, p))

        corr, p = spearmanr(cos_sims, wup_sims )
        print('Spearmans correlation: %.3f, p-value: %s'  % (corr, p))

        plt.scatter(wup_sims, cos_sims)
        plt.title("Wordnet similarity of homonymous senses plotted against cosine similarity of predicted vectors of two tokens in semantic feature space")
        plt.xlabel("Wu and Palmer Similarity")
        plt.ylabel("Cosine Similarity")
        plt.show()


        print()
        print()
        
"""
We have these values for a single word. we cant store them for the whole dataset because, 
even if we only have 1000 senses of a word,
1000 choose 2 leaves us with almost 500K pairwise combinations of tokens.

"""


In [ ]:
"""

some choices here: we are looking at correlations for multi-prototype model
including monosemous words
"""

all_sense_similarities = pd.DataFrame.from_records(all_sense_similarities, columns = ["cos_sim", "wup_sim"])

cos_sims = all_sense_similarities['cos_sim']
wup_sims = all_sense_similarities['wup_sim']

corr, p = pearsonr(cos_sims, wup_sims )
print('Pearsons correlation: %.3f, p-value: %s'  % (corr, p))

corr, p = spearmanr(cos_sims, wup_sims )
print('Spearmans correlation: %.3f, p-value: %s'  % (corr, p))

plt.scatter(wup_sims, cos_sims)
plt.title("Wordnet similarity of homonymous senses plotted against cosine similarity of predicted vectors of two tokens in semantic feature space")
plt.xlabel("Wu and Palmer Similarity")
plt.ylabel("Cosine Similarity")
plt.show()

In [ ]:

# multi_model_predictions = []



# def pluralize(string):
#     if string == 'mouse':
#         return 'mice'
#     else:
#         return string + 's'


    

# models = {"multi_prototype_model_preds": multi_model,
#          #"single_prototype_model_preds": single_model,
#          #"glove_model_preds": glove_model
#          }
# for label, model in models.items():
#     predictions = []        
    
#     for index, row in df.iterrows():
#         if index % 1000 == 0:
#             print("processed %s tokens" % index)
        
#         #print(df.semcor_token)
#         #print(row.sentence_id)
#         sentence = semcor_indexer.get_object(row.sentence_id)
#         #print(sentence)
#         #print(row.semcor_token)
#         prediction = model.predict_in_context(row.semcor_token, sentence, bert)
#         multi_model_predictions.append(prediction)
        
#         #singular = row.semcor_token
#         #plural = engine.plural(singular)

#         try:
#             predicted_vector = model.predict_in_context(row.semcor_token, sentence, bert)
#         except:
#             predicted_vector = model.predict_in_context(row.semcor_token, sentence, bert)


#         predictions.append(predicted_vector)

#     df[label] = predictions

# df
    
# df['multi_prototype_model_predictions'] = predictions

For each pair of tokens of the same lemma, we calculate
    (1) the distance between those two senses in wordnet
    (2) the cosine / jaqqard distance between the two predicted feature vectors.
and store these into a list, along with the lemma.

In [ ]:
"""
this is the stupid way that takes ten decades. why don;t you try this again but smarter in the next cell
"""


# wup_sims = []
# cossine_sims = []

# for index, row in df.iterrows():
#     wordform = row.semcor_token
    
#     # select other examples of this token
#     othertokens = df[df.semcor_token == wordform]
#     # filter out the token itself
#     othertokens = othertokens[othertokens.index != index]
#     #print(othertokens)
    
#     for index, otherword in othertokens.iterrows():
#         # find the wordnet distance between these two wordnet senses
#         synset1 = row.wn_lemma.synset()
#         synset2 = otherword.wn_lemma.synset()
        
#         wup_sim = synset1.wup_similarity(synset2)
#         wup_sims.append(wup_sim)
#         cossim = 1 - cosine(row.multi_prototype_model_preds, otherword.multi_prototype_model_preds)
#         #cossim = 1 - cosine(row.single_prototype_model_preds, otherword.single_prototype_model_preds)
#         cossine_sims.append(cossim)
#         #print(synset1)
#         #print(synset2)
#         #print(wup_sim)
#         #print(cossim)


In [ ]:
"""
same as above cell, but smarter. first we create a list of words in the corpus indexed to all the sentences containing that token.
"""



In [ ]:
# from math import isnan

# print(len(cossine_sims))
# print(len(wup_sims))

# for cs in cossine_sims:
#     if type(cs) != np.float64:
#         print(cs)
        
        
# # get rid of problematic nonetype rows for wu palmer similarity
# # in future I should figure out why they are problematic, giving none for similarity between lemmas


# zipped = list(zip(cossine_sims, wup_sims))
# new_cossine_sims = []
# new_wup_sims = []

# print(len(zipped))

# for index in range(0,len(zipped)):
#     if type(wup_sims[index]) == float:
#         new_cossine_sims.append(cossine_sims[index])
#         new_wup_sims.append(wup_sims[index])

Finally, we correlate the wordnet distance with the predicted features. 